In [1]:
import duckdb
import pandas as pd
import os
import requests

# Chargement des données complete sans 57,67,68

In [2]:
url_complete = "https://www.data.gouv.fr/api/1/datasets/r/cc7b8f0c-45ea-4444-8b55-55d30bc34ac5"
response = requests.get(url_complete)
with open("../data/data_asso/raw/rna_waldec_20250901_complete.parquet", "wb") as f:
    f.write(response.content)

In [3]:
df_asso = duckdb.read_parquet("../data/data_asso/raw/rna_waldec_20250901_complete.parquet")
df_asso

┌────────────┬────────────┬────────────────┬─────────┬─────────┬────────────┬────────────┬────────────┬────────────┬─────────┬────────────┬─────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────┬───────────────┬─────────────────┬──────────────┬─────────────────┬───────────────┬─────────────────────────┬──────────────┬────────────────┬─────────────────┬─────────────────────────┬────────────────┬─────────────────────────┬─────────────────┬─────────────────────┬──────────────┬─────────────────┬──────────────────────┬───────────┬──────────────┬─────────┬──────────┬────────────────────────────────────────────────────────────┬──────────┬─────────────────────

In [7]:
duckdb.sql("""SELECT COUNT(*) AS total_associations FROM df_asso""")

┌────────────────────┐
│ total_associations │
│       int64        │
├────────────────────┤
│            2215247 │
└────────────────────┘

In [102]:
query = """ 
SELECT 
    id,
    adrs_codeinsee, 
    adrs_codepostal
FROM df_asso 
WHERE position = 'A' 
        AND (adrs_codeinsee IS NOT NULL OR adrs_codepostal IS NOT NULL)
        AND (adrs_codeinsee != '0' OR adrs_codepostal != '00000')
        AND (adrs_codeinsee !='0' OR adrs_codepostal IS NOT NULL)
ORDER BY adrs_codeinsee
"""

df_asso_filtered = duckdb.sql(query).df()
df_asso_filtered

,id,adrs_codeinsee,adrs_codepostal
0,W831004501,0,83520
1,W532007664,0,53380
2,W224010165,0,22620
3,W334001988,0,33290
4,W334001929,0,33460
...,...,...,...
1903270,W9N3001346,98833,98818
1903271,W9N3001872,98833,98818
1903272,W9P1003256,INSEE,00000
1903273,W9M3001137,None,97229


In [120]:
df_insee_not_null = duckdb.sql("select * from df_asso_filtered where adrs_codeinsee is not null AND adrs_codeinsee!='0'").df()
df_insee_not_null

,id,adrs_codeinsee,adrs_codepostal
0,W012002746,01001,01400
1,W012000177,01001,01400
2,W012003009,01001,01400
3,W012006848,01001,01400
4,W012003106,01001,01400
...,...,...,...
1903252,W9N3001781,98833,98818
1903253,W9N3000218,98833,98818
1903254,W9N3001346,98833,98818
1903255,W9N3001872,98833,98818


In [103]:
df_insee_null = duckdb.sql("select * from df_asso_filtered where adrs_codeinsee is null or adrs_codeinsee='0'").df()
df_insee_null

,id,adrs_codeinsee,adrs_codepostal
0,W831004501,0,83520
1,W532007664,0,53380
2,W224010165,0,22620
3,W334001988,0,33290
4,W334001929,0,33460
5,W611002119,0,61700
6,W303000479,0,30140
7,W323001391,0,32290
8,W711005777,0,71150
9,W012005428,0,01800


In [104]:

for _ , row in df_insee_null.iterrows():
    code_postal = row['adrs_codepostal']
    code_insee = df_com[df_com['code_postal']==code_postal]['code_insee'].values[0]
    print(f"Code postal: {code_postal} -> Code insee: {code_insee}")
    row['adrs_codeinsee'] = code_insee




Code postal: 83520 -> Code insee: 83107
Code postal: 53380 -> Code insee: 53086
Code postal: 22620 -> Code insee: 22210
Code postal: 33290 -> Code insee: 33056
Code postal: 33460 -> Code insee: 33010
Code postal: 61700 -> Code insee: 61021
Code postal: 30140 -> Code insee: 30010
Code postal: 32290 -> Code insee: 32001
Code postal: 71150 -> Code insee: 71051
Code postal: 01800 -> Code insee: 01054
Code postal: 78870 -> Code insee: 78043
Code postal: 51130 -> Code insee: 51049
Code postal: 80560 -> Code insee: 80003
Code postal: 25210 -> Code insee: 25042
Code postal: 08000 -> Code insee: 08040
Code postal: 78380 -> Code insee: 78092
Code postal: 97229 -> Code insee: 97231
Code postal: 25390 -> Code insee: 25161


In [105]:
df_insee_null

,id,adrs_codeinsee,adrs_codepostal
0,W831004501,83107,83520
1,W532007664,53086,53380
2,W224010165,22210,22620
3,W334001988,33056,33290
4,W334001929,33010,33460
5,W611002119,61021,61700
6,W303000479,30010,30140
7,W323001391,32001,32290
8,W711005777,71051,71150
9,W012005428,01054,01800


In [106]:
query = """ 
select * from df_insee_null

union

select * from df_asso_complete where adrs_codeinsee != '0' order by adrs_codeinsee
"""

df_asso_sans = duckdb.sql(query)
df_asso_sans

┌────────────┬────────────────┬─────────────────┐
│     id     │ adrs_codeinsee │ adrs_codepostal │
│  varchar   │    varchar     │     varchar     │
├────────────┼────────────────┼─────────────────┤
│ W012010953 │ 01001          │ 01400           │
│ W012016589 │ 01001          │ 01400           │
│ W012002201 │ 01001          │ 01400           │
│ W012003106 │ 01001          │ 01400           │
│ W012016171 │ 01001          │ 01400           │
│ W012000467 │ 01001          │ 01400           │
│ W012016503 │ 01001          │ 01400           │
│ W012001710 │ 01001          │ 01400           │
│ W012004101 │ 01001          │ 01400           │
│ W012004615 │ 01001          │ 01400           │
│     ·      │   ·            │   ·             │
│     ·      │   ·            │   ·             │
│     ·      │   ·            │   ·             │
│ W012006270 │ 01235          │ 01240           │
│ W012009878 │ 01235          │ 01240           │
│ W012000783 │ 01235          │ 01240           │


In [124]:
df_sans_pb_postal = duckdb.sql("""SELECT * FROM df_asso_sans where adrs_codepostal not like '00%' order by adrs_codeinsee""").df()
df_sans_pb_postal

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,id,adrs_codeinsee,adrs_codepostal
0,W012003009,01001,01400
1,W012007810,01001,01400
2,W012004101,01001,01400
3,W012016171,01001,01400
4,W012001322,01001,01400
...,...,...,...
1903204,W9N3001320,98833,98818
1903205,W9N3005205,98833,98818
1903206,W9N3000824,98833,98818
1903207,W9N3000756,98833,98818


In [109]:
df_pb_postal = duckdb.sql("""SELECT * FROM df_asso_sans where adrs_codepostal like '00%' order by adrs_codeinsee""")
df_pb_postal.df()

,id,adrs_codeinsee,adrs_codepostal
0,W052007591,05061,00000
1,W372019363,37261,00000
2,W382002700,38530,00000
3,W461007000,46014,00000
4,W491012530,49020,00000
5,W691076647,69383,00000
6,W751266337,75112,00200
7,W751236683,75112,00200
8,W751236756,75112,00200
9,W751277637,75112,00200


In [110]:
query = """ 
SELECT 
    id, 
    '75056' AS adrs_codeinsee,
    '75012' AS adrs_codepostal
FROM df_pb_postal 
WHERE adrs_codeinsee = '75112' and adrs_codepostal like '00%'
ORDER BY adrs_codeinsee
"""

df_paris_corrige = duckdb.sql(query).df()
df_paris_corrige

,id,adrs_codeinsee,adrs_codepostal
0,W751234441,75056,75012
1,W751227385,75056,75012
2,W751236386,75056,75012
3,W751236669,75056,75012
4,W751236761,75056,75012
5,W751236992,75056,75012
6,W751236995,75056,75012
7,W751266337,75056,75012
8,W751266571,75056,75012
9,W751280331,75056,75012


In [114]:
df_pb_postal = duckdb.sql("""SELECT * FROM df_pb_postal where adrs_codeinsee != '75112'""").df()
df_pb_postal

,id,adrs_codeinsee,adrs_codepostal
0,W052007591,05061,00000
1,W372019363,37261,00000
2,W382002700,38530,00000
3,W461007000,46014,00000
4,W491012530,49020,00000
5,W691076647,69383,00000
6,W802013690,80077,00000
7,W912015919,91182,00000
8,W922010968,92004,00000
9,W923001863,92012,00000


In [116]:
for _ , row in df_pb_postal.iterrows():
    code_insee = row['adrs_codeinsee']
    try:
        code_postal = df_com[df_com['code_insee']==code_insee]['code_postal'].values[0]
    except IndexError:
        print(f"Code insee: {code_insee} not found in df_com")
        continue
    print(f"Code postal: {code_insee} -> Code insee: {code_postal}")
    row['adrs_codepostal'] = code_postal

Code postal: 05061 -> Code insee: 05000
Code postal: 37261 -> Code insee: 37100
Code postal: 38530 -> Code insee: 38090
Code insee: 46014 not found in df_com
Code postal: 49020 -> Code insee: 49070
Code insee: 69383 not found in df_com
Code postal: 80077 -> Code insee: 80260
Code insee: 91182 not found in df_com
Code postal: 92004 -> Code insee: 92600
Code postal: 92012 -> Code insee: 92100
Code postal: 92012 -> Code insee: 92100
Code postal: 92023 -> Code insee: 92140
Code postal: 97231 -> Code insee: 97229
Code insee: INSEE not found in df_com


In [117]:
df_pb_postal

,id,adrs_codeinsee,adrs_codepostal
0,W052007591,05061,05000
1,W372019363,37261,37100
2,W382002700,38530,38090
3,W461007000,46014,00000
4,W491012530,49020,49070
5,W691076647,69383,00000
6,W802013690,80077,80260
7,W912015919,91182,00000
8,W922010968,92004,92600
9,W923001863,92012,92100


In [122]:
df_pb_postal_clean = df_pb_postal[df_pb_postal['adrs_codepostal'] != '00000']
df_pb_postal_clean

,id,adrs_codeinsee,adrs_codepostal
0,W052007591,05061,05000
1,W372019363,37261,37100
2,W382002700,38530,38090
4,W491012530,49020,49070
6,W802013690,80077,80260
8,W922010968,92004,92600
9,W923001863,92012,92100
10,W923004987,92012,92100
11,W921005431,92023,92140
12,W9M3001141,97231,97229


In [126]:
df_asso_sans_final = pd.concat([df_sans_pb_postal,df_pb_postal_clean, df_paris_corrige]).sort_values(by='adrs_codeinsee').reset_index(drop=True)
df_asso_sans_final

,id,adrs_codeinsee,adrs_codepostal
0,W012003009,01001,01400
1,W012003106,01001,01400
2,W012002201,01001,01400
3,W012006848,01001,01400
4,W012000177,01001,01400
...,...,...,...
1903257,W9N3001150,98833,98818
1903258,W9N3005349,98833,98818
1903259,W9N3001768,98833,98818
1903260,W9N3005372,98833,98818


# Chargement des fichiers pour la moselle, bas-rhin, haut-rhin

In [127]:
dico_url = {
    "57" : "https://www.data.gouv.fr/api/1/datasets/r/f5073265-9689-441c-bd6d-8d9fbd360161",
    "67" : "https://www.data.gouv.fr/api/1/datasets/r/b7acf7a2-1480-465e-b02b-22633d0a378d",
    "68" : "https://www.data.gouv.fr/api/1/datasets/r/b7d6b412-5da6-4ed2-97cc-c9d8e7b321de"
}

df_temp = pd.DataFrame()
for dept, url in dico_url.items():
    response = requests.get(url)
    with open(f"../data/data_asso/raw/asso_{dept}.csv", "wb") as f:
        f.write(response.content)
    df_dept = pd.read_csv(f"../data/data_asso/raw/asso_{dept}.csv", sep=';',dtype=str)
    print(f"Dept {dept} : {df_dept.shape[0]} rows")
    df_temp = pd.concat([df_temp, df_dept], ignore_index=True)

Dept 57 : 26918 rows
Dept 67 : 40696 rows
Dept 68 : 24647 rows


In [128]:
df_temp['ETAT_ASSOCIATION'].unique()

array(['INSCRITE', 'DISSOUTE', 'RADIEE', 'NONCONFORME'], dtype=object)

In [129]:
df_temp

,NUMERO_AMALIA,TRIBUNAL,DENOMINATION,ETAT_ASSOCIATION,OBJET,ADRESSE_POSTALE,COMMUNE,CODE_POSTAL,COMPLEMENT_ADRESSE,DOMAINES,...,BUT,SIREN - SIRET,DUREE_VIE,DATE_STATUTS_INITIAUX,DATE_DECLARATION_INSCRIPTION,DATE_INSCRIPTION,DATE_STATUTS_EN_VIGUEUR,DATE_DISSOLUTION,DATE_PUBLICATION_DISSOLUTION,DATE_RADIATION
0,A2002THI000075,Tribunal judiciaire de THIONVILLE,MADAGASCAR TERRE DE LIBERTE,INSCRITE,"La défense, l'unification de Madagascar, le so...","1 rue de la Gare,",KOENIGSMACKER,57970,8,[SOCIAL-SOLIDARITE],...,NON_LUCRATIF,NaN,NaN,2002-04-01,NaN,2002-11-03,2002-04-01,NaN,NaN,NaN
1,A2000MET000251,Tribunal judiciaire de METZ,SUPPORTER CLUB DU RUGBY CLUB DE METZ MOSELLE,INSCRITE,Soutenir le club de Rugby par tous les moyens,Complexe de Rugby de la Grange aux Bois - Rue ...,METZ,57070,NaN,[SPORTS],...,NON_LUCRATIF,NaN,NaN,2000-12-05,NaN,2000-12-26,2000-12-05,NaN,NaN,NaN
2,A1983THI000002,Tribunal judiciaire de THIONVILLE,"""CLUB DE L'AMITIE""",INSCRITE,"Créer, gérer, animer un groupe du 3e âge.","Mairie, 1 rue de l'église",BOUSSE,57310,NaN,[SOCIAL-SOLIDARITE],...,NON_LUCRATIF,NaN,NaN,1983-09-20,NaN,1983-09-19,2022-01-26,NaN,NaN,NaN
3,A1990SAG000051,Tribunal judiciaire de SARREGUEMINES,"Club des Amis du ""SKAT"" à BITCHE",INSCRITE,Pratique et éducation du jeu de cartes du SKAT.,Maison des Associations - Glacis du Château,BITCHE,57230,NaN,[LOISIRS-NATURE],...,NON_LUCRATIF,NaN,NaN,1990-03-30,NaN,1990-05-24,2016-02-20,NaN,NaN,NaN
4,A1969MET000019,Tribunal judiciaire de METZ,UNION DEPARTEMENTALE DES ASSOCIATIONS,INSCRITE,L'association a pour objet de donner son avis ...,RUE ROYAL CANADIAN AIR FORCE ars laquenexy,METZ,57000,15179,[SOCIAL-SOLIDARITE],...,NON_LUCRATIF,NaN,NaN,1967-04-09,NaN,1969-03-24,2023-06-26,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92256,A1997SEL000067,Tribunal de proximité de SELESTAT,LES ACOLYTES,INSCRITE,L'association LES ACOLYTES fondée en 1996 a po...,168 rue Clémenceau,SAINTE-MARIE-AUX-MINES,68160,NaN,[DIVERS],...,NON_LUCRATIF,NaN,NaN,1996-10-08,NaN,1997-06-01,1996-10-08,NaN,NaN,NaN
92257,A1983MUL000024,Tribunal judiciaire de MULHOUSE,GYMNASTIQUE D'ENTRETIEN POUR ADULTES,RADIEE,La pratique de la gymnastique volontaire d'ent...,Maison des Associations,WITTENHEIM,68270,NaN,[Sport-Gymnastique d'entretien],...,NON_LUCRATIF,NaN,NaN,1983-09-27,NaN,1983-12-06,1988-10-12,NaN,NaN,2017-03-15
92258,A1955THA000004,Tribunal de proximité de THANN,SKI CLUB EDELWEISS SAINT AMARIN,INSCRITE,diffuser entre tous les membres les techniques...,Mairie,SAINT-AMARIN,68550,NaN,[SPORTS],...,NON_LUCRATIF,NaN,NaN,1954-10-07,NaN,1955-04-03,1954-10-07,NaN,NaN,NaN
92259,A1997SEL000062,Tribunal de proximité de SELESTAT,COMITE DE SOUTIEN AU PARC MINIER,INSCRITE,L'association a pour but la défense du Projet ...,20 rue d'Untergrombach,SAINTE-MARIE-AUX-MINES,68160,NaN,[DIVERS],...,NON_LUCRATIF,NaN,NaN,1997-06-07,NaN,1997-11-16,1997-06-07,NaN,NaN,NaN


In [130]:
query = """ 
SELECT 
NUMERO_AMALIA as id,
COMMUNE as commune,
CODE_POSTAL as adrs_codepostal
FROM df_temp
WHERE ETAT_ASSOCIATION = 'INSCRITE'
ORDER BY CODE_POSTAL
"""

df_asso_manquant = duckdb.sql(query)
df_asso_manquant

┌────────────────┬─────────────────────┬─────────────────┐
│       id       │       commune       │ adrs_codepostal │
│    varchar     │       varchar       │     varchar     │
├────────────────┼─────────────────────┼─────────────────┤
│ A1979MET000059 │ LE BAN-SAINT-MARTIN │ 57              │
│ A2010MET000274 │ METZ                │ 5700            │
│ A2025MET000156 │ METZ                │ 57000           │
│ A2020MET000220 │ METZ                │ 57000           │
│ A2003MET000207 │ METZ                │ 57000           │
│ A2003MET000088 │ METZ                │ 57000           │
│ A1969MET000021 │ METZ                │ 57000           │
│ A1975MET000065 │ METZ                │ 57000           │
│ A2025MET000205 │ METZ                │ 57000           │
│ A2001MET000086 │ METZ                │ 57000           │
│       ·        │  ·                  │   ·             │
│       ·        │  ·                  │   ·             │
│       ·        │  ·                  │   ·            

In [131]:
# ligne dont adrs_codepostal à 5 caractères
df_asso_manquant_5 = duckdb.sql("""
SELECT * FROM df_asso_manquant
WHERE LENGTH(adrs_codepostal) != 5
""")
df_asso_manquant_5

┌────────────────┬─────────────────────┬─────────────────┐
│       id       │       commune       │ adrs_codepostal │
│    varchar     │       varchar       │     varchar     │
├────────────────┼─────────────────────┼─────────────────┤
│ A1979MET000059 │ LE BAN-SAINT-MARTIN │ 57              │
│ A2010MET000274 │ METZ                │ 5700            │
│ A2021MET000005 │ METZ                │ 570000          │
│ A1992MET000114 │ HAGONDANGE          │ 573000          │
│ A1965STA000005 │ LELLING             │ 57657660        │
│ A2002STR000084 │ STRASBOURG          │ 6700            │
│ A2019STR000225 │ STRASBOURG          │ 670000          │
│ A2015STR000321 │ STRASBOURG          │ 670000          │
│ A2002COL000038 │ COLMAR              │ 680000          │
│ A2001MUL000207 │ HIRTZBACH           │ 681180          │
│ A1988MUL000120 │ DURMENACH           │ 684809          │
│ A2000GUE000016 │ LAUTENBACHZELL      │ 686102          │
├────────────────┴─────────────────────┴────────────────

In [133]:
mapping_code_postal = {
    "57" : "57050",
    "5700": "57000",
    "570000": "57000",
    "573000": "57300",
    "57657660": "57660",
    "67000": "67000",
    "670000": "67000",
    "680000": "68000",
    "681180 ": "68118",
    "684809": "68480",
    "686102": "68610",
    }

df_asso_manquant_corrige = df_asso_manquant.df().replace({"adrs_codepostal": mapping_code_postal})
df_asso_manquant_corrige.sort_values(by="adrs_codepostal")

,id,commune,adrs_codepostal
2455,A2012MET000028,METZ,57000
1979,A2015MET000237,METZ,57000
1980,A2023MET000205,METZ,57000
1981,A2002MET000216,METZ,57000
1982,A2024MET000224,METZ,57000
...,...,...,...
75347,A2004MUL000095,HEIMSBRUNN,68990
75349,A2009MUL000068,HEIMSBRUNN,68990
75350,A2019MUL000216,HEIMSBRUNN,68990
75352,A2025MUL000012,HEIMSBRUNN,68990


# Chargement des communes pour jointures

In [134]:
df_com = pd.read_csv("../data/raw/communes_france_2025.csv", dtype=str)

In [135]:
df_com

,Unnamed: 0,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,...,longitude_mairie,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,niveau_equipements_services,niveau_equipements_services_texte,gentile,url_wikipedia,url_villedereve
0,0,01001,L'Abergement-Clémenciat,Abergement-Clémenciat,à Abergement-Clémenciat,de l'Abergement-Clémenciat,l-abergement-clemenciat,L'ABERGEMENT-CLÉMENCIAT,COM,commune,...,4.921,46.153,4.926,6,Rural à habitat dispersé,0.0,communes non pôle,NaN,https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01001-l-abergemen...
1,1,01002,L'Abergement-de-Varey,Abergement-de-Varey,à Abergement-de-Varey,de l'Abergement-de-Varey,l-abergement-de-varey,L'ABERGEMENT-DE-VAREY,COM,commune,...,5.423,46.009,5.428,6,Rural à habitat dispersé,0.0,communes non pôle,"Abergementais, Abergementaises",https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01002-l-abergemen...
2,2,01004,Ambérieu-en-Bugey,Ambérieu-en-Bugey,à Ambérieu-en-Bugey,d'Ambérieu-en-Bugey,amberieu-en-bugey,AMBÉRIEU-EN-BUGEY,COM,commune,...,5.36,45.961,5.373,2,Centres urbains intermédiaires,3.0,centres structurants d'équipements et de services,"Ambarrois, Ambarroises",https://fr.wikipedia.org/wiki/fr:Ambérieu-en-B...,https://villedereve.fr/ville/01004-amberieu-en...
3,3,01005,Ambérieux-en-Dombes,Ambérieux-en-Dombes,à Ambérieux-en-Dombes,d'Ambérieux-en-Dombes,amberieux-en-dombes,AMBÉRIEUX-EN-DOMBES,COM,commune,...,4.903,45.996,4.912,5,Bourgs ruraux,1.0,centres locaux d'équipements et de services,Ambarrois,https://fr.wikipedia.org/wiki/fr:Ambérieux-en-...,https://villedereve.fr/ville/01005-amberieux-e...
4,4,01006,Ambléon,Ambléon,à Ambléon,d'Ambléon,ambleon,AMBLÉON,COM,commune,...,5.601,45.75,5.594,6,Rural à habitat dispersé,0.0,communes non pôle,Ambléonais,https://fr.wikipedia.org/wiki/fr:Ambléon,https://villedereve.fr/ville/01006-ambleon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34930,34930,97613,M'Tsangamouji,M'Tsangamouji,à M'Tsangamouji,de M'Tsangamouji,m-tsangamouji,M'TSANGAMOUJI,COM,commune,...,45.084,-12.751,45.087,3,Petites villes,NaN,NaN,NaN,https://fr.wikipedia.org/wiki/fr:M'Tsangamouji,https://villedereve.fr/ville/97613-m-tsangamouji
34931,34931,97614,Ouangani,Ouangani,à Ouangani,d'Ouangani,ouangani,OUANGANI,COM,commune,...,45.139,-12.837,45.138,3,Petites villes,NaN,NaN,NaN,https://fr.wikipedia.org/wiki/fr:Ouangani,https://villedereve.fr/ville/97614-ouangani
34932,34932,97615,Pamandzi,Pamandzi,à Pamandzi,de Pamandzi,pamandzi,PAMANDZI,COM,commune,...,45.275,-12.796,45.284,2,Centres urbains intermédiaires,NaN,NaN,Pamandziens,https://fr.wikipedia.org/wiki/fr:Pamandzi,https://villedereve.fr/ville/97615-pamandzi
34933,34933,97616,Sada,Sada,à Sada,de Sada,sada,SADA,COM,commune,...,45.106,-12.861,45.119,2,Centres urbains intermédiaires,NaN,NaN,Sadois,https://fr.wikipedia.org/wiki/fr:Sada (Mayotte),https://villedereve.fr/ville/97616-sada


## Jointure

In [136]:
query_join = """ 
SELECT 
    a.id,
    c.nom_standard_majuscule as commune,
    a.adrs_codepostal,
    c.code_insee as adrs_codeinsee  
FROM df_asso_manquant_corrige a
LEFT JOIN df_com c
ON a.commune = c.nom_standard_majuscule AND LEFT(a.adrs_codepostal, 2) = LEFT(c.code_postal, 2)
ORDER BY a.adrs_codepostal  
"""

df_joined = duckdb.sql(query_join)
df_joined

┌────────────────┬───────────────────┬─────────────────┬────────────────┐
│       id       │      commune      │ adrs_codepostal │ adrs_codeinsee │
│    varchar     │      varchar      │     varchar     │    varchar     │
├────────────────┼───────────────────┼─────────────────┼────────────────┤
│ A2025MET000235 │ METZ              │ 57000           │ 57463          │
│ A2009MET000053 │ METZ              │ 57000           │ 57463          │
│ A2007MET000133 │ METZ              │ 57000           │ 57463          │
│ A1982MET000135 │ METZ              │ 57000           │ 57463          │
│ A2003MET000174 │ METZ              │ 57000           │ 57463          │
│ A2016MET000016 │ METZ              │ 57000           │ 57463          │
│ A1989MET000120 │ METZ              │ 57000           │ 57463          │
│ A2010MET000153 │ METZ              │ 57000           │ 57463          │
│ A1978MET000112 │ METZ              │ 57000           │ 57463          │
│ A1979MET000064 │ METZ              │

In [137]:
#Vérification de la taille du dataframe
df_joined.df().shape

(75383, 4)

# Association des deux dataframe

In [138]:
query = """ 
SELECT 
    id,
    adrs_codeinsee,
    adrs_codepostal 
FROM df_joined

UNION

SELECT * 
FROM df_asso_sans_final
ORDER BY adrs_codeinsee
"""

df_asso_complete = duckdb.sql(query)
df_asso_complete

┌────────────┬────────────────┬─────────────────┐
│     id     │ adrs_codeinsee │ adrs_codepostal │
│  varchar   │    varchar     │     varchar     │
├────────────┼────────────────┼─────────────────┤
│ W012016503 │ 01001          │ 01400           │
│ W012004101 │ 01001          │ 01400           │
│ W012001322 │ 01001          │ 01400           │
│ W012016589 │ 01001          │ 01400           │
│ W012000177 │ 01001          │ 01400           │
│ W012000467 │ 01001          │ 01400           │
│ W012002201 │ 01001          │ 01400           │
│ W012001710 │ 01001          │ 01400           │
│ W012007810 │ 01001          │ 01400           │
│ W012003106 │ 01001          │ 01400           │
│     ·      │   ·            │   ·             │
│     ·      │   ·            │   ·             │
│     ·      │   ·            │   ·             │
│ W012006616 │ 01235          │ 01240           │
│ W012005211 │ 01235          │ 01240           │
│ W012008337 │ 01235          │ 01240           │


# Jointure avec df_epci

In [145]:
data_epci = pd.read_csv("../data/data_asso/raw/data_epci.csv", sep=";",encoding="latin1")
data_epci

,dept,siren,raison_sociale,nature_juridique,mode_financ,nb_membres,total_pop_tot,total_pop_mun,dep_com,insee,siren_membre,nom_membre,ptot_2025,pmun_2025
0,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01304,210103040.0,Pont-d'Ain,2 912,2 862
1,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01199,210101994.0,Jujurieux,2 246,2 209
2,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01363,210103636.0,Saint-Jean-le-Vieux,1 873,1 799
3,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01314,210103149.0,Priay,1 826,1 803
4,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01273,210102737.0,Neuville-sur-Ain,1 823,1 798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34867,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B138,212001382.0,Lavatoggio,158,156
34868,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B296,212002968.0,Sant'Antonino,133,132
34869,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B153,212001531.0,Manso,120,117
34870,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B025,212000251.0,Avapessa,81,81


In [152]:
query = """ 
SELECT 
    TRY_CAST(siren AS INTEGER) AS siren,
    raison_sociale,
    dept,
    insee,
    TRY_CAST(REPLACE(total_pop_tot, ' ', '') AS INTEGER) AS total_pop_tot,
from data_epci
"""

df_epci = duckdb.sql(query)
df_epci

┌───────────┬────────────────────────────────────┬─────────┬─────────┬───────────────┐
│   siren   │           raison_sociale           │  dept   │  insee  │ total_pop_tot │
│   int32   │              varchar               │ varchar │ varchar │     int32     │
├───────────┼────────────────────────────────────┼─────────┼─────────┼───────────────┤
│ 200029999 │ CC Rives de l'Ain - Pays du Cerdon │ 01      │ 01304   │         15156 │
│ 200029999 │ CC Rives de l'Ain - Pays du Cerdon │ 01      │ 01199   │         15156 │
│ 200029999 │ CC Rives de l'Ain - Pays du Cerdon │ 01      │ 01363   │         15156 │
│ 200029999 │ CC Rives de l'Ain - Pays du Cerdon │ 01      │ 01314   │         15156 │
│ 200029999 │ CC Rives de l'Ain - Pays du Cerdon │ 01      │ 01273   │         15156 │
│ 200029999 │ CC Rives de l'Ain - Pays du Cerdon │ 01      │ 01303   │         15156 │
│ 200029999 │ CC Rives de l'Ain - Pays du Cerdon │ 01      │ 01068   │         15156 │
│ 200029999 │ CC Rives de l'Ain - Pays du C

In [153]:
query = """

SELECT 
    e2.dept,
    CAST(e2.siren AS VARCHAR) as id_epci, 
    e2.raison_sociale as nom_epci,
    'i131' AS id_indcator,
    ROUND(count(e1.adrs_codeinsee) / e2.total_pop_tot * 1000,2) AS valeur_brute
FROM df_epci e2
LEFT JOIN df_asso_complete e1
ON e1.adrs_codeinsee = e2.insee
GROUP BY e2.dept,e2.siren,e2.total_pop_tot, e2.raison_sociale
ORDER BY dept, siren
"""

df_asso_epci = duckdb.sql(query)
df_asso_epci

┌─────────┬───────────┬─────────────────────────────────────────────────┬─────────────┬──────────────┐
│  dept   │  id_epci  │                    nom_epci                     │ id_indcator │ valeur_brute │
│ varchar │  varchar  │                     varchar                     │   varchar   │    double    │
├─────────┼───────────┼─────────────────────────────────────────────────┼─────────────┼──────────────┤
│ 01      │ 200029999 │ CC Rives de l'Ain - Pays du Cerdon              │ i131        │        29.03 │
│ 01      │ 200040350 │ CC Bugey Sud                                    │ i131        │        28.85 │
│ 01      │ 200042497 │ CC Dombes Saône Vallée                          │ i131        │        26.35 │
│ 01      │ 200042935 │ CA Haut-Bugey Agglomération                     │ i131        │        21.37 │
│ 01      │ 200069193 │ CC de la Dombes                                 │ i131        │        32.85 │
│ 01      │ 200070118 │ CC Val de Saône Centre                          │

In [154]:
#Sauvegarde du résultat final
df_asso_epci.write_csv("../data/data_asso/processed/i131_asso_per_epci.csv")